In [1]:
import numpy as np

import time
import mujoco
import mujoco.viewer

import mujoco
import numpy as np
import matplotlib.pyplot as plt
import mujoco.viewer

import mediapy as media
import time

import matplotlib.pyplot as plt
from scipy.linalg import null_space
from scipy.spatial.transform import Rotation as R

In [2]:
from main_redundancy_sim import LeapNodeMujoco, GraspClass, PosRot, GradientDescentIK, OnlyPosIK

model_path = '/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/leaphand_redundancy.xml'
leap_hand = LeapNodeMujoco(model_path)

IK_index=OnlyPosIK('/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/redundancy/0_index_sim.xml')
IK_thumb=OnlyPosIK('/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/redundancy/0_thumb_sim.xml')
# IK_thumb_pos=OnlyPosIK('/home/sysidea/leap_hand_mujoco/model/leap hand/thumb.xml')
IK_middle=OnlyPosIK('/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/middle.xml')
IK_tertiary=OnlyPosIK('/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/tertiary.xml')

In [3]:
camera1 = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(leap_hand.m, camera1)

# Set camera position
camera1.lookat[:] = np.array([0.0, 0.0, 0.0])  # Point in space the camera is looking at
camera1.azimuth = -90                # Horizontal angle in degrees
camera1.elevation = -10                      # Vertical angle in degrees
camera1.distance = 1                         # Distance from the lookat point

camera2 = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(leap_hand.m, camera2)

# Set camera position
camera2.lookat[:] = np.array([0.0, 0.0, 0.0])  # Point in space the camera is looking at
camera2.azimuth = 0               # Horizontal angle in degrees
camera2.elevation = -90                      # Vertical angle in degrees
camera2.distance = 1                         # Distance from the lookat point

camera3 = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(leap_hand.m, camera3)

# Set camera position
camera3.lookat[:] = np.array([0.0, 0.0, 0.15])  # Point in space the camera is looking at
camera3.azimuth = 90               # Horizontal angle in degrees
camera3.elevation = -15                      # Vertical angle in degrees
camera3.distance = 1                         # Distance from the lookat point


In [4]:
mujoco.mj_step(leap_hand.m,leap_hand.d)

pos_worldi1=leap_hand.d.site(leap_hand.m.site('contact_index1').id).xpos.reshape(3)
rot_worldi1=leap_hand.d.site(leap_hand.m.site('contact_index1').id).xmat.reshape(3,3)
pos_worldt1=leap_hand.d.site(leap_hand.m.site('contact_thumb1').id).xpos.reshape(3)
rot_worldt1=leap_hand.d.site(leap_hand.m.site('contact_thumb1').id).xmat.reshape(3,3)

pos_worldi2=leap_hand.d.site(leap_hand.m.site('contact_index2').id).xpos.reshape(3)
rot_worldi2=leap_hand.d.site(leap_hand.m.site('contact_index2').id).xmat.reshape(3,3)
pos_worldt2=leap_hand.d.site(leap_hand.m.site('contact_thumb2').id).xpos.reshape(3)
rot_worldt2=leap_hand.d.site(leap_hand.m.site('contact_thumb2').id).xmat.reshape(3,3)

pos_world_palm=leap_hand.d.body(leap_hand.m.body('palm').id).xpos.reshape(3)
rot_world_palm=np.reshape(leap_hand.d.body(leap_hand.m.body('palm').id).xmat,[3,3])

pos_palmi1=pos_worldi1-pos_world_palm
rot_palmi1=rot_world_palm.T @ rot_worldi1
pos_palmt1=pos_worldt1-pos_world_palm
rot_palmt1=rot_world_palm.T @ rot_worldt1

pos_palmi2=pos_worldi2-pos_world_palm
rot_palmi2=rot_world_palm.T @ rot_worldi2
pos_palmt2=pos_worldt1-pos_world_palm
rot_palmt2=rot_world_palm.T @ rot_worldt2

Rpk_index = np.reshape(leap_hand.d.body(leap_hand.m.body('mcp_joint').id).xmat,[3,3])

Rpk_thumb = np.reshape(leap_hand.d.body(leap_hand.m.body('pip_4').id).xmat,[3,3])

qd_d=leap_hand.d
qd_m=leap_hand.m


In [5]:
result_index1=IK_index.calculate(pos_palmi1,'contact_index')
result_thumb1=IK_thumb.calculate(pos_palmt1,'contact_thumb')
result_index2=IK_index.calculate(pos_palmi2, 'contact_index')
result_thumb2=IK_thumb.calculate(pos_palmt2, 'contact_thumb')

print("result_index1",result_index1)
print("result_thumb1",result_thumb1)
print("result_index2",result_index2)
print("result_thumb2",result_thumb2)


framerate=30

#go near the contact point
while leap_hand.d.time < 2:

    

    #placeholder code 
    #index finger
    [a,b,c,d]=result_index1
    [e,f,g,h]=result_thumb1
    
    leap_hand.apply_controls_hand([a,b,c,d,0,0,0,0,0,0,0,0,e,f,g,h])
    
    #temperory
    leap_hand.step_video(framerate,camera1)

while leap_hand.d.time < 4 and leap_hand.d.time > 2:

    

    #placeholder code 
    #index finger
    [a,b,c,d]=result_index2
    [e,f,g,h]=result_thumb2
    
    leap_hand.apply_controls_hand([a,b,c,d,0,0,0,0,0,0,0,0,e,f,g,h])
    
    #temperory
    
    leap_hand.step_video(framerate,camera1)

leap_hand.play_video(framerate)

result_index1 [1.2289728  0.4137102  0.33806647 0.04604031]
result_thumb1 [ 1.78873561  0.15355999 -0.10329076 -0.02914892]
result_index2 [1.32722948 0.41038796 0.33538345 0.02513859]
result_thumb2 [ 1.78873561  0.15355999 -0.10329076 -0.02914892]
Total frames captured: 121


This browser does not support the video tag.


In [6]:
grasp=GraspClass()
index_path='/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/index_finger.xml'


index_J=grasp.J(index_path,'contact_index',leap_hand.d.qpos[0:4])
print(index_J)

[[ 3.57706679e-02 -4.96231010e-03  3.69895717e-02  3.36943302e-02]
 [ 0.00000000e+00 -8.60686564e-02 -1.77480391e-03 -1.60455136e-03]
 [-1.24484585e-01  4.31753743e-05 -7.15314312e-02 -3.55825453e-02]
 [ 0.00000000e+00 -8.70033096e-03  4.71347933e-02  4.71347933e-02]
 [ 1.00000000e+00  0.00000000e+00  9.98888454e-01  9.98888454e-01]
 [ 0.00000000e+00 -9.99962151e-01 -4.10103823e-04 -4.10103823e-04]]


In [7]:
Rpk_index=Rpk_index
Rpk_thumb=Rpk_thumb

b1 = np.array([0.028, 0, 0]).reshape(3, 1)
b2 = np.array([-0.028, 0, 0]).reshape(3, 1)

index_path='/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/index_finger.xml'
thumb_path='/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/thumb.xml'


# pos_worldi2=leap_hand.d.site(leap_hand.m.site('contact_index2').id).xpos.reshape(3)
# rot_worldi2=leap_hand.d.site(leap_hand.m.site('contact_index2').id).xmat.reshape(3,3)

while leap_hand.d.time < 10 and leap_hand.d.time > 4:
    grasp = GraspClass()

    n = 2
    contact1 = leap_hand.d.site(leap_hand.m.site('contact_index2').id).xmat.reshape(3,3)
    contact2 = leap_hand.d.site(leap_hand.m.site('contact_thumb2').id).xmat.reshape(3,3)

    #from mujoco xml file


    J1 = index_J=grasp.J(index_path,'contact_index',leap_hand.d.qpos[-16:-12])
    J2 = index_J=grasp.J(index_path,'contact_index',leap_hand.d.qpos[-4:])

    contact_orientations = [contact1, contact2]
    Rpks = [Rpk_index, Rpk_thumb]
    Js = [J1, J2]
    r_theta = leap_hand.d.body(leap_hand.m.body('cube').id).xmat.reshape(3,3)
    bs = [b1, b2]

    # Compute the grasp matrices
    Jh_ = grasp.Jh_full(n, contact_orientations, Rpks, Js)
    G_ = grasp.G_full(n, contact_orientations, r_theta, bs)

    H=grasp.selection_matrix(n,'SF')

    Jh_leap=H@Jh_
    G_leap_T=H@G_.T
    G_leap=G_leap_T.T

    # Controller parameters
    Kp_d = 0.1*np.eye(6)
    Kp_k = 1

    n0 = 1000*np.ones([8,1])
    I = np.eye(8)
    phi_d = np.ones([8,1])

#leap_hand = LeapNode_Taucontrol()

# Main control loop
    q_d = leap_hand.d
    q_m = leap_hand.m

    posrot=PosRot()

    q_final=posrot.q_subs(qd_m,q_m,qd_d,q_d,'cube')

    # Compute forces
    Fimp = np.dot(np.linalg.pinv(G_leap),np.dot(Kp_d , (q_final.reshape(6,1))))
    Fnull = (I - np.matmul(np.linalg.pinv(G_leap), G_leap)) @ n0
    
    # Compute desired torque
    Tau_dy = Jh_leap.T @ (Fimp + Fnull)
    
    
    
    # Read position from the Leap hand
    #phi = leap_hand.read_pos()
    phi=np.ones([8,1])
    
    # Kinematic control torque
    Tau_kin = Kp_k * (phi_d - phi)  # Corrected this line
    
    # Total torque
    Tau = Tau_dy + Tau_kin*0

    leap_hand.d.ctrl[0:4]=Tau[0:4].reshape(4)
    leap_hand.d.ctrl[-4:]=Tau[-4:].reshape(4)

    
    leap_hand.step_video(framerate,camera1)
    print("Current time:", leap_hand.d.time) 

leap_hand.play_video(framerate)


Current time: 4.001999999999671
Current time: 4.0029999999996715
Current time: 4.003999999999672
Current time: 4.004999999999672
Current time: 4.0059999999996725
Current time: 4.006999999999673
Current time: 4.007999999999673
Current time: 4.0089999999996735
Current time: 4.009999999999674
Current time: 4.010999999999674
Current time: 4.0119999999996745
Current time: 4.012999999999675
Current time: 4.013999999999675
Current time: 4.0149999999996755
Current time: 4.015999999999676
Current time: 4.016999999999676
Current time: 4.0179999999996765
Current time: 4.018999999999677
Current time: 4.019999999999677
Current time: 4.0209999999996775
Current time: 4.021999999999678
Current time: 4.022999999999678
Current time: 4.0239999999996785
Current time: 4.024999999999679
Current time: 4.025999999999679
Current time: 4.0269999999996795
Current time: 4.02799999999968
Current time: 4.02899999999968
Current time: 4.0299999999996805
Current time: 4.030999999999681
Current time: 4.031999999999681


KeyboardInterrupt: 